# Toy example for Radix-based Benders

In [1]:
%load_ext line_profiler

In [2]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f4b92492d88>)


### Try optimizing using radix for one condition first

In [74]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
keff0 = 1./65/3600
crowding_bound = 0.0003
not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [75]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x

In [76]:
opt = Optimizer(ijomc)
gap = opt.add_duality_gap_constraint(INF=1e6)

In [77]:
gap.optimize()

<Solution 1.47 at 0x7f4bb596f9d0>

In [78]:
import numpy as np

radix = 2.
powers = np.arange(-3,4)
print(powers)
digits_per_power = radix
pwr_max = max(powers)
digits = np.linspace(0, radix-1, digits_per_power)
print(digits)

# Discretize crowding coefficients into radix
crowding_p = gap.metabolites.crowding

var_cons_dict = {}
for rxn_p in crowding_p.reactions:
    # Get the coefficient in the dual
    var_d = gap.reactions.wa_crowding
    cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]
    a0 = rxn_p.metabolites[crowding_p]    
    var_cons_dict[rxn_p.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

[-3 -2 -1  0  1  2  3]
[0. 1.]


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [79]:
# %lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

In [80]:
%time opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

CPU times: user 3.1 s, sys: 47.7 ms, total: 3.14 s
Wall time: 4.13 s


In [81]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary'):
    rxn.objective_coefficient = 1.

In [82]:
N_CONDS = 1

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r.id for r in ijomc.reactions.query('EX_')]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [107]:
mu_meas = 0.74

F_TOL = 0.005

gap.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = mu_meas*(1-F_TOL)
gap.reactions.BIOMASS_Ecoli_core_w_GAM.upper_bound = mu_meas*(1+F_TOL)

from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

In [108]:
milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 5545 rows, 2374 columns and 14186 nonzeros
Variable types: 1702 continuous, 672 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-01, 1e+06]
  RHS range        [3e-04, 1e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1621 rows and 239 columns
Presolve time: 0.10s
Presolved: 3924 rows, 2135 columns, 10574 nonzeros
Variable types: 1477 continuous, 658 integer (658 binary)

Root relaxation: objective 3.698998e-01, 1374 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.36990    0    3          -    0.36990      -     -    0s
     0     0    0.36990    0    3          -    0.36990  

In [109]:
#gap.optimize('minimize')
rxn_mu =gap.reactions.BIOMASS_Ecoli_core_w_GAM 
#muopt = rxn_mu
sol = gurobi_solver.format_solution(milp, gap)
muopt = sol.x_dict[rxn_mu.id]
yopt = [sol.x_dict[rxn.id] for rxn in gap.reactions.query('binary_')]
sum(yopt)
print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))
print('Number of non-zero binaries: %g' % sum(yopt))

Initial mu_crowd=0.736887. Fitted within 0.5%: 0.7363 <= 0.736727 <= 0.7437
Number of non-zero binaries: 4


In [110]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol.x_dict[yid]
            if abs(y)>1e-10:
                print yid, y
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

binary_TPI14 1.0
binary_TPI16 1.0
binary_ENO16 1.0
binary_PFK13 1.0


In [111]:
kfit_dict

{u'ACALD': 0.0,
 u'ACKr': 0.0,
 u'ACONTa': 0.0,
 u'ACONTb': 0.0,
 u'ADK1': 0.0,
 u'AKGDH': 0.0,
 u'ALCD2x': 0.0,
 u'CS': 0.0,
 u'ENO': 3.418803418803419e-05,
 u'FBA': 0.0,
 u'FBP': 0.0,
 u'FRD7': 0.0,
 u'FUM': 0.0,
 u'G6PDH2r': 0.0,
 u'GAPD': 0.0,
 u'GLNS': 0.0,
 u'GLUDy': 0.0,
 u'GLUN': 0.0,
 u'GLUSy': 0.0,
 u'GND': 0.0,
 u'ICDHyr': 0.0,
 u'ICL': 0.0,
 u'LDH_D': 0.0,
 u'MALS': 0.0,
 u'MDH': 0.0,
 u'ME1': 0.0,
 u'ME2': 0.0,
 u'NADTRHD': 0.0,
 u'PDH': 0.0,
 u'PFK': 4.273504273504274e-06,
 u'PFL': 0.0,
 u'PGI': 0.0,
 u'PGK': 0.0,
 u'PGL': 0.0,
 u'PGM': 0.0,
 u'PPC': 0.0,
 u'PPCK': 0.0,
 u'PPS': 0.0,
 u'PTAr': 0.0,
 u'PYK': 0.0,
 u'RPE': 0.0,
 u'RPI': 0.0,
 u'SUCDi': 0.0,
 u'SUCOAS': 0.0,
 u'TALA': 0.0,
 u'TKT1': 0.0,
 u'TKT2': 0.0,
 u'TPI': 4.2735042735042735e-05}

### Plug back in to be sure

In [112]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding_bound = ijomc.metabolites.crowding._bound
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

In [113]:
ijofit.optimize()

<Solution 0.74 at 0x7f4bb596f150>